In [7]:
import urllib
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#SCRAPING FUNCTIONS 

def raw_scrape(link):    
    driver.get(link)
    for i in range(1,6):
        try:
            driver.find_element_by_xpath('/html/body/div['+str(i)+']/div[1]/button/span[1]').click()
            break
        except:
            pass
   
    lineup = list()
    time.sleep(1)
    for k in range(1,3):
        for j in range(1,15):
            try:
                font = driver.find_element_by_xpath('//*[@id="tabpagesmatch"]/div[1]/div['+str(k)+']/div/div/div/div[1]/div[1]/div/table/tbody/tr['+str(j)+']/td[3]/a/strong')
                name = driver.find_element_by_xpath('//*[@id="tabpagesmatch"]/div[1]/div['+str(k)+']/div/div/div/div[1]/div[1]/div/table/tbody/tr['+str(j)+']/td[3]/a').text
                #print(j)
                space = name.find(" ")
                name = name[0]+ "."+ name[space:]
                lineup.append(name)
            except:
                continue
    
    driver.find_element_by_xpath('//*[@id="tabbedmatch"]/ul/li[2]/a').click()
    output = list()
    
    j = 0

    time.sleep(6)
    for i in range(1,6):
        j = 0
        while True:
            entry = list()
            j = j + 1
            try:
                home_action = driver.find_element_by_xpath('//*[@id="mCSB_5_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[1]').text
                times = driver.find_element_by_xpath('//*[@id="mCSB_5_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[3]').text
                away_action = driver.find_element_by_xpath('//*[@id="mCSB_5_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[5]').text
                home_score = driver.find_element_by_xpath('//*[@id="mCSB_5_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[2]').text
                away_score = driver.find_element_by_xpath('//*[@id="mCSB_5_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[4]').text
                output.append([times,home_action,away_action,home_score,away_score])
            except:
                break
            
    fin_output = [lineup,output]
    return fin_output

def raw_to_pbp(fin_output):
    quarter = 1
    newrow = [0, "Pregame",0,0,0,0]
    for player in fin_output[0]:
        newrow.append(player)
        
        
    raw = fin_output[1]
    pbp_fin = list()
    pbp_fin.append(newrow.copy())
    quarter = 1
    for i in range(len(raw)):
        
        newrow[0] = time_to_secs(raw[i][0],quarter)
        
        if raw[i][1] != "":
            newrow[1] = raw[i][1]
        else:
            newrow[1] = raw[i][2]
        
        if "koniec części gry" in raw[i][1]:
            quarter += 1
        
        
        if raw[i][3] != " ":
            newrow[2] = int(raw[i][3])
            newrow[3] = int(raw[i][4])
            newrow[4] = int(raw[i][3]) - int(raw[i][4])
            newrow[5]
            
        if "zejście" in raw[i][1]:
            comma_1 = raw[i][1].find(",")
            comma_2 = raw[i][1][comma_1+1:].find(",")
            off = raw[i][1][comma_1+2:comma_2+comma_1+1]
            if off[-1] == " ":
                off = off[0:len(off)-1]
            if off[0] == " ":
                off = off[1:len(off)]
            for l in range(6,11):
                if newrow[l] == off:
                    newrow[l] = " "
                    break
        
        elif "wejście" in raw[i][1]:
            comma_1 = raw[i][1].find(",")
            comma_2 = raw[i][1][comma_1+1:].find(",")
            on = raw[i][1][comma_1+2:comma_2+comma_1+1]
            if on[-1] == " ":
                on = on[0:len(on)-1]
            if on[0] == " ":
                on = on[1:len(on)]
            for l in range(6,11):
                if newrow[l] == " ":
                    newrow[l] = on
                    break
        
        if "zejście" in raw[i][2]:
            comma_1 = raw[i][2].find(",")
            comma_2 = raw[i][2][comma_1+1:].find(",")
            off = raw[i][2][comma_1+2:comma_2+comma_1+1]
            if off[-1] == " ":
                off = off[0:len(off)-1]
            if off[0] == " ":
                off = off[1:len(off)]
            for l in range(11,16):
                if newrow[l] == off:
                    newrow[l] = " "
                    break
        
        elif "wejście" in raw[i][2]:
            comma_1 = raw[i][2].find(",")
            comma_2 = raw[i][2][comma_1+1:].find(",")
            on = raw[i][2][comma_1+2:comma_2+comma_1+1]
            if on[-1] == " ":
                on = on[0:len(on)-1]
            if on[0] == " ":
                on = on[1:len(on)]
            for l in range(11,16):
                if newrow[l] == " ":
                    newrow[l] = on
                    break
        pbp_fin.append(newrow.copy())
    return pbp_fin[:len(pbp_fin)-1]
                                 

#HELPER FUNCTION

def time_to_secs(time,quarter):
    timeleft = int(time[0:2])*60 + int(time[3:5]) + int(time[6:8])*0.1
    return (600 - timeleft)+(quarter-1)*600



#LINEUP EVALUATION FUNCTIONS

def lineup_summary(game_pbp):
    lineupz = list()
    output = list()
    for i in range(0,len(game_pbp)):
        if transition(game_pbp) == False:
            if [sorted(game_pbp[i][6:11]),sorted(game_pbp[i][11:16])] not in lineupz:
                lineupz.append([sorted(game_pbp[i][6:11]),sorted(game_pbp[i][11:16])])
    for lineup in lineupz:
        summ = 0
        time = 0
        for i in range(1,len(game_pbp)):
            if [sorted(game_pbp[i][6:11]),sorted(game_pbp[i][11:16])] == lineup:
                summ += (game_pbp[i][4] - game_pbp[i-1][4])
                time += (game_pbp[i][0] - game_pbp[i-1][0])
                if transition(game_pbp[i]) == True:
                    time += (game_pbp[i+1][0] - game_pbp[i][0])
        if time != 0:
            output.append([time,summ,lineup])
    return output


def transition(pbp_entry):
    for j in range(6,16):
             if pbp_entry[j] == " ":
                return True
    return False


def find_link(linkz,game):
    for i in range(len(linkz)):
        if game in linkz[i]:
            return linkz[i]
        

def find_link_index(linkz,game):
    for i in range(len(linkz)):
        if game in linkz[i]:
            return i  
        
def ovrtm_raw_scrape(link):    
    driver.get(link)
    for i in range(1,6):
        try:
            driver.find_element_by_xpath('/html/body/div['+str(i)+']/div[1]/button/span[1]').click()
            break
        except:
            pass
   
    lineup = list()
    time.sleep(1)
    for k in range(1,3):
        for j in range(1,15):
            try:
                font = driver.find_element_by_xpath('//*[@id="tabpagesmatch"]/div[1]/div['+str(k)+']/div/div/div/div[1]/div[1]/div/table/tbody/tr['+str(j)+']/td[3]/a/strong')
                name = driver.find_element_by_xpath('//*[@id="tabpagesmatch"]/div[1]/div['+str(k)+']/div/div/div/div[1]/div[1]/div/table/tbody/tr['+str(j)+']/td[3]/a').text
                #print(j)
                space = name.find(" ")
                name = name[0]+ "."+ name[space:]
                lineup.append(name)
            except:
                continue
    
    driver.find_element_by_xpath('//*[@id="tabbedmatch"]/ul/li[2]/a').click()
    output = list()
    
    j = 0

    time.sleep(6)
    for i in range(1,6):
        j = 0
        while True:
            entry = list()
            j = j + 1
            try:
                home_action = driver.find_element_by_xpath('//*[@id="mCSB_6_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[1]').text
                times = driver.find_element_by_xpath('//*[@id="mCSB_6_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[3]').text
                away_action = driver.find_element_by_xpath('//*[@id="mCSB_6_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[5]').text
                home_score = driver.find_element_by_xpath('//*[@id="mCSB_6_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[2]').text
                away_score = driver.find_element_by_xpath('//*[@id="mCSB_6_container"]/div/table/tbody['+str(i)+']/tr['+str(j)+']/td[4]').text
                output.append([times,home_action,away_action,home_score,away_score])
            except:
                break
            
    fin_output = [lineup,output]
    return fin_output


def as_string(lineup):
    output = lineup[0]
    for i in range(1,len(lineup)):
        output = output + ", " + str(lineup[i])
    return output

def same_lineup(lp1, lp2):
    for name in lp1:
         if name not in lp2:
                return False
    return True


In [4]:
#AFTER OPENING DOC

import dbm
import pickle
db = dbm.open('PolandBball','c')
pchdr = db["plkresults_p1"]
db.close()
plk_games = pickle.loads(pchdr)

db = dbm.open('PolandBball','c')
pchdr1 = db["plkresults_p2"]
db.close()
plk_games_2 = pickle.loads(pchdr1)

db = dbm.open('PolandBball','c')
pchdrlink = db["plklinks"]
db.close()
links = pickle.loads(pchdrlink)

db = dbm.open('PolandBball','c')
pchdr2 = db["plkresults_p3"]
db.close()
plk_games_3 = pickle.loads(pchdr2)

db = dbm.open('PolandBball','c')
pchdr3 = db["plkresults_p4"]
db.close()
plk_games_4 = pickle.loads(pchdr3)

db = dbm.open('PolandBball','c')
pchdr4 = db["plkresults_p5"]
db.close()
plk_games_5 = pickle.loads(pchdr4)

db = dbm.open('PolandBball','c')
pchdr5 = db["plkresults_p6"]
db.close()
plk_games_6 = pickle.loads(pchdr5)

db = dbm.open('PolandBball','c')
pchdr6 = db["plk_lp_comp"]
db.close()
lp_comp_plk20_char = pickle.loads(pchdr6)

db = dbm.open('PolandBball','c')
pchdr7 = db["graph"]
db.close()
graph = pickle.loads(pchdr7)

#db = dbm.open('PolandBball','c')
#pchdr7 = db["node2vec_plk"]
#db.close()
#node2vec_plk = pickle.loads(pchdr7)

In [5]:
#BEFORE CLOSING DOC ALWAYS

import dbm
import pickle
db = dbm.open('PolandBball', 'c')

pchdr = pickle.dumps(plk_games)
db["plkresults_p1"] = pchdr


pchdr1 = pickle.dumps(plk_games_2)
db["plkresults_p2"] = pchdr1

pchdrlink = pickle.dumps(links)
db["plklinks"] = pchdrlink

pchdr2 = pickle.dumps(plk_games_3)
db["plkresults_p3"] = pchdr2


pchdr3 = pickle.dumps(plk_games_4)
db["plkresults_p4"] = pchdr3

pchdr4 = pickle.dumps(plk_games_5)
db["plkresults_p5"] = pchdr4

pchdr5 = pickle.dumps(plk_games_6)
db["plkresults_p6"] = pchdr5

pchdr6 = pickle.dumps(lp_comp_plk20_char)
db["plk_lp_comp"] = pchdr6

pchdr7 = pickle.dumps(graph)
db["graph"] = pchdr7

#pchdr7 = pickle.dumps(node2vec_plk)
#db["node2vec_plk"] = pchdr7

db.close()

In [ ]:
#EXAMPLE FINDING LINKS FOR SEASON
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(executable_path = "/Users/pietervantol/Documents/chromedriver", options = options)
link = "https://www.plk.pl/terminarz-i-wyniki.html"
i = 1

driver.get(link)
pkl_results = list()
links = list()

for a in driver.find_elements_by_xpath('.//a'):
    q_link = a.get_attribute('href')
    if "mecz" in q_link and q_link not in links:
        links.append(a.get_attribute('href'))
len(links)

In [18]:
#EXAMPLE SCRAPING PBPS FROM MULTIPLE GAMES 
options = webdriver.ChromeOptions()
#options.add_argument('headless') 
driver = webdriver.Chrome(executable_path = "/Users/pietervantol/Documents/chromedriver", options = options)
#plk_games_6 = list()
for i in range(240):
    if i not in scraped:
        try:
            pbp = raw_to_pbp(raw_scrape(links[i]))
            plk_games_6.append(links[i][31:links[i].find(".html")])
            plk_games_6.append(pbp)
            print(i)
        except:
            try:
                #print("hi")
                pbp = raw_to_pbp(ovrtm_raw_scrape(links[i]))
                plk_games_6.append(links[i][31:links[i].find(".html")])
                plk_games_6.append(pbp)
                print(i,"overtime")
            except:
                pass


13
15
16


In [151]:
#lp_comp_plk20_char = list()
nextrow = list()
for i in range(len(plk_games_6)):
    if i % 2 != 0:
        lp_summary = lineup_summary(plk_games_6[i])
        for j in range(len(lp_summary)):
            home = as_string(lp_summary[j][2][0])
            away = as_string(lp_summary[j][2][1])
            margin = (lp_summary[j][1]/lp_summary[j][0])*60
            nextrow = [home,away,margin]
            lp_comp_plk20_char.append(nextrow.copy())

In [174]:
deleted = list()
new_comp_plk20_char = list()
for i in range(len(lp_comp_plk20_char)):
    if i in deleted:
        continue
    else:
        newrowz = lp_comp_plk20_char[i]
        for j in range(len(lp_comp_plk20_char)):
            if lp_comp_plk20_char[i][0] == lp_comp_plk20_char[j][0] and lp_comp_plk20_char[i][1] == lp_comp_plk20_char[j][1] and i != j:
                newrowz[2] += lp_comp_plk20_char[j][2]
                deleted.append(j)
                print(j)
            elif lp_comp_plk20_char[i][0] == lp_comp_plk20_char[j][1] and lp_comp_plk20_char[i][1] == lp_comp_plk20_char[j][0] and i != j:
                newrowz[2] = newrowz[2] - lp_comp_plk20_char[j][2] 
                deleted.append(j)
                print(j,"reverse fixture")
        new_comp_plk20_char.append(newrowz)
                
                

In [152]:
#ALL LINEUP MATCHUPS PKL 2020 SEASON
lp_comp_plk20_char

[['A. Pluta, I. Almeida, M. Moore, S. Jones, W. Lichodiej',
  'K. Borowski, K. Łączyński, M. Dziemba, M. Laksa, R. Szymański',
  1.075268817204301],
 ['A. Pluta, I. Almeida, M. Moore, S. Jones, W. Lichodiej',
  'D. Jeszke, J. Sharma, K. Łączyński, M. Dziemba, M. Laksa',
  -1.5789473684210527],
 ['A. Pluta, I. Almeida, I. Radić, M. Moore, W. Lichodiej',
  'D. Jeszke, J. Sharma, K. Łączyński, M. Dziemba, M. Laksa',
  8.333333333333334],
 ['I. Almeida, I. Radić, M. Moore, P. Zamojski, W. Lichodiej',
  'D. Jeszke, J. Sharma, K. Łączyński, M. Laksa, Y. Franke',
  3.571428571428571],
 ['I. Almeida, I. Radić, M. Moore, P. Zamojski, W. Lichodiej',
  'B. Pelczar, D. Jeszke, J. Sharma, M. Dziemba, Y. Franke',
  0.0],
 ['I. Radić, K. Sulima, M. Moore, P. Zamojski, R. Clarke',
  'B. Pelczar, D. Jeszke, J. Sharma, M. Dziemba, Y. Franke',
  -0.43478260869565216],
 ['I. Almeida, I. Radić, K. Sulima, P. Zamojski, R. Clarke',
  'B. Pelczar, D. Jeszke, J. Sharma, M. Dziemba, Y. Franke',
  2.376237623762